In [1]:
import os
os.environ["PROJ_LIB"] = os.path.join(os.environ["CONDA_PREFIX"], "share", "proj")
import pandas as pd
from bs4 import BeautifulSoup as soup
import cfscrape
import json
import csv
import networkx as nx
import datetime
import glob
import network_visualizer
from datetime import datetime, timedelta
import time
import numpy as np

In [2]:
from folium import Map
from folium.plugins import HeatMap

In [3]:
airports_df = pd.read_csv('airports_data/flight_radar_24_airports.csv', sep=',')

european_airports_df = pd.read_csv('airports_data/european_airports.csv', sep=',')

delays_df = pd.read_csv('delays/departure_arival_delays21_Jul_2020.csv', sep=',')

In [4]:
european_airport_codes_list = european_airports_df['IATA'].tolist()

In [ ]:
airports_df

In [ ]:
delays_df

In [ ]:
departures_delays_list = []
arrivals_delays_list = []

airports_dict = {}
for index, row in airports_df.iterrows():
    airports_dict[row['Label']] = (row['latitude'], row['longitude'])

In [ ]:
airports_dict

In [ ]:
for index, row in delays_df.iterrows():
    if row['departure_delay_list']:
        if (row['from_airports_list'] in airports_dict.keys()) and (row['from_airports_list'] in european_airport_codes_list):
            lat, longt = airports_dict[row['from_airports_list']]
            departures_delays_list.append((lat, longt, abs(float(row['departure_delay_time_list']))))

    if row['arrival_delay_list']:
        if (row['to_airports_list'] in airports_dict.keys()) and (row['to_airports_list'] in european_airport_codes_list):
            lat, longt = airports_dict[row['to_airports_list']]
            arrivals_delays_list.append((lat, longt, abs(float(row['arrival_delay_time_list']))))

In [ ]:
departures__map = Map(location=[48.499998, 23.3833318], zoom_start=4, )

In [ ]:
departures_layer = HeatMap(departures_delays_list,
                  min_opacity=0.9,
                  radius=17, 
                  blur=25, 
                  max_zoom=5,
)

In [ ]:
departures__map.add_child(departures_layer)

In [ ]:
arrivals__map = Map(location=[48.499998, 23.3833318], zoom_start=4, )

In [ ]:
arrivals_layer = HeatMap(arrivals_delays_list,
                  min_opacity=0.2,
                  radius=17, 
                  blur=15, 
                  max_zoom=1,
)

In [ ]:
arrivals__map.add_child(departures_layer)

In [ ]:
#just calculate the average departure and arrival delay of every airport and then create the heatmaps again

In [ ]:
average_departure_delay_per_airport = {}
average_arrival_delay_per_airport = {}

In [ ]:
# for every airport gather all the departure and arrival delays
for index, row in delays_df.iterrows():
    if row['departure_delay_list']:
        if row['from_airports_list'] in airports_dict.keys() and row['from_airports_list'] in european_airport_codes_list:
            if row['from_airports_list'] not in average_departure_delay_per_airport.keys():
                average_departure_delay_per_airport[row['from_airports_list']] = []
            average_departure_delay_per_airport[row['from_airports_list']].append(abs(float(row['departure_delay_time_list'])))
    if row['arrival_delay_list']:
        if row['to_airports_list'] in airports_dict.keys() and row['to_airports_list'] in european_airport_codes_list:
            if row['to_airports_list'] not in average_arrival_delay_per_airport.keys():
                average_arrival_delay_per_airport[row['to_airports_list']] = []
            average_arrival_delay_per_airport[row['to_airports_list']].append(abs(float(row['arrival_delay_time_list'])))

In [ ]:
average_departures_delays_per_coordinates = []
for key, delays_list in average_departure_delay_per_airport.items():
    if key in airports_dict.keys():
        lat, longt = airports_dict[key]
        average_departures_delays_per_coordinates.append((lat, longt, np.mean(delays_list)))

In [ ]:
average_arrivals_delays_per_coordinates = []
for key, delays_list in average_arrival_delay_per_airport.items():
    if key in airports_dict.keys():
        lat, longt = airports_dict[key]
        average_arrivals_delays_per_coordinates.append((lat, longt, np.mean(delays_list)))

In [ ]:
len(average_departures_delays_per_coordinates)

In [ ]:
average_departures_delays_per_coordinates

In [ ]:
max([i[2] for i in average_departures_delays_per_coordinates])

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.hist([i[2] for i in average_departures_delays_per_coordinates], 5)

In [ ]:
average_departures__map = Map(location=[48.499998, 23.3833318], zoom_start=4, )

average_departures_layer = HeatMap(average_departures_delays_per_coordinates,
                  max_val=max([i[2] for i in average_departures_delays_per_coordinates]),
                  min_opacity=0.2,
                  radius=20, 
                  blur=1, 
                  max_zoom=1,
)

In [ ]:
average_departures__map.add_child(average_departures_layer)

In [ ]:
plt.hist([i[2] for i in average_arrivals_delays_per_coordinates], 5)

In [ ]:
average_arrivals__map = Map(location=[48.499998, 23.3833318], zoom_start=4, )

average_arrivals_layer = HeatMap(average_arrivals_delays_per_coordinates,
                  max_val=max([i[2] for i in average_arrivals_delays_per_coordinates]),
                  min_opacity=0.2,
                  radius=20, 
                  blur=1, 
                  max_zoom=1,
)

In [ ]:
average_arrivals__map.add_child(average_arrivals_layer)

In [5]:
canceled_flights_df = pd.read_csv('canceled_flights/canceled_flights_on_24_Jul_2020.csv', sep=',')

In [6]:
canceled_flights_df

,Unnamed: 0,canceled_departure_list,canceled_arrival_list,canceled_flight_codes_list
0,0,EVN,SHJ,G9244
1,1,SHJ,EVN,G9245
2,2,VIE,SOF,FB410
3,3,SOF,VIE,FB409
4,4,VIE,SHJ,G9574
...,...,...,...,...
178,178,DUB,LHR,EI168
179,179,LHR,HND,BA5
180,180,LHR,HND,JL42
181,181,HND,LHR,JL41


In [14]:
average_canceled_flight = {}

In [15]:
airports_dict = {}
for index, row in airports_df.iterrows():
    airports_dict[row['Label']] = (row['latitude'], row['longitude'])

In [16]:
# for every airport gather all the departure and arrival delays
for index, row in canceled_flights_df.iterrows():
    if row['canceled_departure_list'] in airports_dict.keys() and row['canceled_departure_list'] in european_airport_codes_list:
        if row['canceled_departure_list'] not in average_canceled_flight.keys():
            average_canceled_flight[row['canceled_departure_list']] = 0
        average_canceled_flight[row['canceled_departure_list']] += 1

In [17]:
average_canceled_flight

{'EVN': 1,
 'VIE': 3,
 'SOF': 4,
 'GYD': 3,
 'BRU': 1,
 'SJJ': 2,
 'DBV': 2,
 'MUC': 2,
 'ZAG': 2,
 'LCA': 1,
 'CPH': 6,
 'MRS': 1,
 'MPL': 1,
 'NTE': 1,
 'CDG': 10,
 'DUB': 4,
 'TBS': 4,
 'WAW': 2,
 'CGN': 2,
 'HAM': 6,
 'FRA': 3,
 'FDH': 1,
 'HHN': 2,
 'KIV': 2,
 'MXP': 2,
 'BUD': 2,
 'ALA': 2,
 'ZRH': 2,
 'AMS': 10,
 'SVO': 4,
 'LIS': 2,
 'GVA': 3,
 'DME': 4,
 'LED': 1,
 'ARN': 1,
 'IST': 10,
 'SAW': 2,
 'TZX': 3,
 'KBP': 1,
 'LHR': 3}